### import

In [1]:
import pandas as pd
import numpy as np
from os.path import join
from datetime import datetime
import functions
import warnings
warnings.simplefilter("ignore")
deposit_dtype = functions.deposit_dtype

In [ ]:
# ################################
company = "솔림"
최종조회일 = "250731"
데이터종류 = "입금"
# ################################
최초조회일 = "1993" if company == "솔림" else ""
path_excel = join(r"D:\3.자산\전산 dataset",company,"기간축적데이터", 데이터종류,)
path_pkl = join(r"D:\3.자산\전산 dataset",company, "기간축적데이터", 데이터종류, "pkl")
fn_write = company + 데이터종류+"전체_" + 최초조회일 + "~" + 최종조회일 +".pkl"

In [ ]:
# pkl 파일 읽기
df_ori = pd.read_pickle(functions.키워드로파일명찾기(path_pkl, 포함키워드=company+데이터종류+"전체"))


# # 엑셀파일 읽기
# fnlist = functions.키워드로파일명찾기(path_excel, 포함키워드=데이터종류+"조회새창", 여러파일허용=True)

# if isinstance(fnlist, str) : # 파일 하나인 경우
#     df = pd.read_excel(fnlist, dtype=deposit_dtype)
# else : # 파일 여러개(리스트)인 경우
#     df = pd.DataFrame(None)
#     for fn in fnlist :
#         print(f'{fn} 읽기 시작')
#         df = pd.concat([df, pd.read_excel(fn, dtype=deposit_dtype)])
        
# # PKL 저장
# df.to_pickle(join(path_pkl, fn_write))

In [8]:
deposit_ori.to_pickle(join(path_pkl, fn_write))

In [78]:
df_ori = pd.read_excel(r"D:\3.자산\금감원\비정기 요청자료\250715_5천이하 7년이상 상환내역 추가요청\2.솔림_전체_원데이터_250630.xlsx", dtype={"채무자키":str, "계좌키":str}, sheet_name='리스트')

In [7]:
print(deposit_ori.입금기간.unique())
print(deposit_ori.입금자구분.unique())
print(deposit_ori.입금구분.unique())

['매입전' nan '원상회복전' '매각사측' '\x7f' '환불' '매각후' '환매' '회수제외' '']
['채무자' '보증인']
['약정분납' '채권압류' '유체동산' '임의변제' '기타' '개인회생' '신용회복' '부채증명서' '법원환급금' '일반입금'
 '대위변제' '배당금' '부동산경매' '회생폐지' '파산배당' '상속배당' '해지비용' '환매대금']


In [ ]:
print(len(deposit_ori))
# data선별
    # 입금기간
deposit_ori = deposit_ori[~deposit_ori.입금기간.isin(["매입전","매각후","환불","환매","회수제외"])]
    # 입금자구분 : 채무자
deposit_ori = deposit_ori[deposit_ori.입금자구분=="채무자"]
    # 입금구분 : 약정분납, 임의변제, 개인회생, 신용회복, 일반입금, 대위변제, 회생폐지, 파산배당, 상속배당 이 아닌 것 (채무자의 신청으로 인한 입금은 포함됨)
deposit_ori = deposit_ori[deposit_ori.입금구분.isin(["약정분납", "임의변제", "개인회생", "신용회복", "일반입금", "대위변제", "회생폐지", "파산배당", "상속배당"])]
print(len(deposit_ori))

# 칼럼타입 정리
deposit_ori["입금일"] = pd.to_datetime(deposit_ori["입금일"], errors='coerce') # na 없음 확인함

# 정렬 : 입금일 역순으로
deposit_ori = deposit_ori.sort_values(by="입금일", ascending=False)

699277
699277


In [42]:
# 재시작점
deposit = deposit_ori[["주민번호인","계좌키","입금일","입금합계","회수원금","입금원금"]].copy()

In [155]:
# 차주별 연말 현재원금 확인하기
차주별연도별원금 = df_ori.groupby(by='주민번호인')["최초원금"].sum().reset_index()
차주별연도별원금

,주민번호인,최초원금
0,110111-0651425,0
1,160625-1852213,9901126
2,170919-2691810,45000000
3,200306-1915516,1484727
4,210815-2450714,17482867
...,...,...
27225,980804-2522214,5000000
27226,980807-1661210,698632
27227,980827-2381025,5000000
27228,981029-2243319,3287650


In [165]:
# 7년연체 정리 : 가장 오래된 연체일로 적용
연체일 = df_ori.sort_values(by=['주민번호인','7년연체'])[['주민번호인','7년연체']]
연체일 = 연체일.drop_duplicates(subset=['주민번호인'], keep='first')
연체일

,주민번호인,7년연체
18338,110111-0651425,2017-01-01
18201,160625-1852213,2006-12-15
138,170919-2691810,2004-12-02
17400,200306-1915516,1997-12-15
1700,210815-2450714,2004-09-10
...,...,...
34822,980804-2522214,2024-11-05
15900,980807-1661210,2024-12-22
6630,980827-2381025,2024-12-10
34823,981029-2243319,2024-10-31


In [166]:
차주별최종정리 = 차주별연도별원금.merge(연체일)
차주별최종정리

,주민번호인,최초원금,7년연체
0,110111-0651425,0,2017-01-01
1,160625-1852213,9901126,2006-12-15
2,170919-2691810,45000000,2004-12-02
3,200306-1915516,1484727,1997-12-15
4,210815-2450714,17482867,2004-09-10
...,...,...,...
27225,980804-2522214,5000000,2024-11-05
27226,980807-1661210,698632,2024-12-22
27227,980827-2381025,5000000,2024-12-10
27228,981029-2243319,3287650,2024-10-31


In [167]:
# 연도 추출
deposit["연도"] = deposit["입금일"].dt.year

# 연도 필터링 (2017 ~ 2025)
filtered = deposit[deposit["연도"].between(2017, 2025)]

# groupby로 연도 및 주민번호인별 회수원금 합계
result = (
    filtered
    .groupby(["연도", "주민번호인"])["회수원금"]
    .sum()
    .reset_index()
    .rename(columns={"회수원금": "연도별회수원금합계"})
)


In [168]:
pivot = result.pivot(index="주민번호인", columns="연도", values="연도별회수원금합계").fillna(0)
pivot

연도,2017,2018,2019,2020,2021,2022,2023,2024,2025
주민번호인,,,,,,,,,
000105-3158019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000106-4157712,0.0,0.0,0.0,0.0,0.0,80000.0,760000.0,680000.0,0.0
000107-3685525,0.0,0.0,0.0,0.0,0.0,531560.0,3189360.0,956808.0,0.0
000125-4106314,0.0,0.0,0.0,0.0,0.0,30456816.0,86294312.0,105420691.0,0.0
000207-4559916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
991215-2561913,0.0,0.0,0.0,0.0,37008.0,135696.0,506661.0,1189566.0,0.0
991220-2908610,0.0,0.0,0.0,0.0,231510.0,4398690.0,9916345.0,2006420.0,0.0
991227-2590411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
merged = pd.merge(차주별최종정리, pivot, on="주민번호인", how="left")
# pivot.columns = ["주민번호인","최초원금"] + [f"{col}" for col in pivot.columns[1:]]
merged = merged.fillna(0)
merged

,주민번호인,최초원금,7년연체,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,110111-0651425,0,2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,160625-1852213,9901126,2006-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,170919-2691810,45000000,2004-12-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,200306-1915516,1484727,1997-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,210815-2450714,17482867,2004-09-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27225,980804-2522214,5000000,2024-11-05,0.0,0.0,0.0,0.0,95580.0,2070900.0,4285170.0,6324210.0,461970.0
27226,980807-1661210,698632,2024-12-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27227,980827-2381025,5000000,2024-12-10,0.0,0.0,0.0,0.0,0.0,4389.0,1078671.0,383466.0,0.0
27228,981029-2243319,3287650,2024-10-31,0.0,0.0,0.0,0.0,1367820.0,9301176.0,0.0,0.0,0.0


In [178]:
# 연도별5000이하여부
def 연도별오천여부_7년연체여부(row) : 
    if row.최초원금 <= 50000000 : 
        rst = []
        for year in range(2017,2025) : 
            if row["7년연체"] <= datetime(year,12,31) :
                rst.append(True)
                break
            else : 
                rst.append(False)
    else : 
        rst = []
        기초원금 = row.최초원금
        for year in range(2017, 2025) : 
            기초원금 -= row.get(year, 0)
            if 기초원금 <= 50000000 :
                if row["7년연체"] <= datetime(year,12,31) :
                    rst.append(True)
                    break
                else : 
                    rst.append(False)
            else : 
                rst.append(False)
                
        # 혹시라도 누락된 경우 채워주기
    while len(rst) < 8:
        rst.append(rst[-1] if rst else False)
        
    return rst
    

In [179]:
cols = ["집계대상" + str(i) for i in range(2018, 2026)]
merged[cols] = merged.apply(lambda row : pd.Series(연도별오천여부_7년연체여부(row)), axis=1)

In [180]:
merged

,주민번호인,최초원금,7년연체,2017,2018,2019,2020,2021,2022,2023,2024,2025,집계대상2018,집계대상2019,집계대상2020,집계대상2021,집계대상2022,집계대상2023,집계대상2024,집계대상2025
0,110111-0651425,0,2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True,True,True,True
1,160625-1852213,9901126,2006-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True,True,True,True
2,170919-2691810,45000000,2004-12-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True,True,True,True
3,200306-1915516,1484727,1997-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True,True,True,True
4,210815-2450714,17482867,2004-09-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27225,980804-2522214,5000000,2024-11-05,0.0,0.0,0.0,0.0,95580.0,2070900.0,4285170.0,6324210.0,461970.0,False,False,False,False,False,False,False,True
27226,980807-1661210,698632,2024-12-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,True
27227,980827-2381025,5000000,2024-12-10,0.0,0.0,0.0,0.0,0.0,4389.0,1078671.0,383466.0,0.0,False,False,False,False,False,False,False,True
27228,981029-2243319,3287650,2024-10-31,0.0,0.0,0.0,0.0,1367820.0,9301176.0,0.0,0.0,0.0,False,False,False,False,False,False,False,True


In [181]:
# 회수합계

# 연도 필터링 (2017 ~ 2025)
filtered1 = deposit[deposit["연도"].between(2018, 2025)]

# groupby로 연도 및 주민번호인별 회수원금 합계
result1 = (
    filtered1
    .groupby(["연도", "주민번호인"])["입금합계"]
    .sum()
    .reset_index()
    .rename(columns={"입금합계": "연도별입금합계"})
)

pivot1 = result1.pivot(index="주민번호인", columns="연도", values="연도별입금합계").fillna(0)
pivot1



연도,2018,2019,2020,2021,2022,2023,2024,2025
주민번호인,,,,,,,,
000105-3158019,0.0,0.0,0.0,358884.0,0.0,0.0,0.0,0.0
000106-4157712,0.0,0.0,0.0,0.0,40000.0,80000.0,48000.0,0.0
000107-3685525,0.0,0.0,0.0,0.0,637872.0,531560.0,106312.0,0.0
000125-4106314,0.0,0.0,0.0,390472.0,4685664.0,4682952.0,4265332.0,0.0
000207-4559916,0.0,0.0,0.0,0.0,0.0,3000000.0,0.0,0.0
...,...,...,...,...,...,...,...,...
991215-2561913,0.0,0.0,0.0,24672.0,24672.0,154923.0,154275.0,300000.0
991220-2908610,0.0,0.0,0.0,154340.0,463020.0,463020.0,1755534.0,0.0
991227-2590411,0.0,0.0,0.0,0.0,4000000.0,0.0,0.0,0.0


In [184]:
# 2018-하반기 회수합계

# 연도 필터링 (2017 ~ 2025)
filtered2 = deposit[(deposit["입금일"]>=datetime(2018,7,1)) & (deposit["입금일"]<=datetime(2018,12,31))]

# groupby로 연도 및 주민번호인별 회수원금 합계
result2 = (
    filtered2
    .groupby("주민번호인")["입금합계"]
    .sum()
    .reset_index()
    .rename(columns={"입금합계": "2018-2"})
)



In [185]:
result2

,주민번호인,2018-2
0,380105-2036219,500000
1,381119-2023118,9200000
2,381212-1829519,500000
3,430909-1101410,1300000
4,440622-1041116,520000
...,...,...
190,800602-1235024,249444
191,800622-2912015,1850000
192,801029-1663222,5000000
193,810425-1560350,1900000


In [ ]:
x = merged.drop(columns=["최초원금","7년연체", 2017,	2018,	2019,2020,	2021,	2022,	2023,	2024,	2025])
x

In [186]:
rst_merged = pd.merge(x, pivot1, on="주민번호인", how="left")
# pivot.columns = ["주민번호인","최초원금"] + [f"{col}" for col in pivot.columns[1:]]
rst_merged = rst_merged.fillna(0)
rst_merged

,주민번호인,집계대상2018,집계대상2019,집계대상2020,집계대상2021,집계대상2022,집계대상2023,집계대상2024,집계대상2025,2018,2019,2020,2021,2022,2023,2024,2025
0,110111-0651425,True,True,True,True,True,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,160625-1852213,True,True,True,True,True,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,170919-2691810,True,True,True,True,True,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,200306-1915516,True,True,True,True,True,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,210815-2450714,True,True,True,True,True,True,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27225,980804-2522214,False,False,False,False,False,False,False,True,0.0,0.0,0.0,63720.0,207090.0,191160.0,223020.0,598940.0
27226,980807-1661210,False,False,False,False,False,False,False,True,0.0,0.0,1164.0,37054.0,0.0,0.0,0.0,0.0
27227,980827-2381025,False,False,False,False,False,False,False,True,0.0,0.0,0.0,0.0,20001.0,171732.0,31224.0,0.0
27228,981029-2243319,False,False,False,False,False,False,False,True,0.0,0.0,0.0,683910.0,5096411.0,0.0,0.0,0.0


In [187]:
rst_merged.to_excel(r"D:\3.자산\금감원\비정기 요청자료\250715_5천이하 7년이상 상환내역 추가요청\차주별연도별입금액.xlsx", index=False)

In [190]:
result2 = result2.merge(rst_merged[["주민번호인","집계대상2018"]], how='left')

In [191]:
result2.to_excel(r"D:\3.자산\금감원\비정기 요청자료\250715_5천이하 7년이상 상환내역 추가요청\차주별연도별입금액_2018하반기.xlsx", index=False)

In [192]:
연도들 = list(range(2018, 2026))

결과 = []
for year in 연도들:
    col_flag = f"집계대상{year}"
    col_amt = year

    # 조건 1: 5천이하YYYY가 True인 행들의 해당 연도 금액 합계
    합계 = rst_merged.loc[rst_merged[col_flag] == True, col_amt].sum()

    # 조건 2: 해당 연도 금액이 0보다 큰 행 개수
    건수 = ((rst_merged[col_flag] == True) & (rst_merged[col_amt] > 0)).sum()

    결과.append((year, 합계, 건수))

# 결과를 보기 좋게 DataFrame으로
df_결과 = pd.DataFrame(결과, columns=["연도", "5천이하합계", "0초과건수"])


In [193]:
df_결과.to_excel(r"D:\3.자산\금감원\비정기 요청자료\250715_5천이하 7년이상 상환내역 추가요청\연도별요약.xlsx", index=False)

In [ ]:
# 2018년 ~ 2025년
def 연도별정리(deposit) : 
    for year in range(2018, 2026) : 
        sub_df = deposit[(datetime(year, 1,1) <= deposit.입금일) & (deposit.입금일 <= datetime(year, 12,31))]
        # 기간내 최초 현재원금 합계 구하고 5000만원 초과인 것은 제외하기
        최초입금 = 


_2018 = (deposit.입금일>=pd.to_datetime('2018-01-01')) & (deposit.입금일<=pd.to_datetime('2018-12-31'))
_2019 = (deposit.입금일>=pd.to_datetime('2019-01-01')) & (deposit.입금일<=pd.to_datetime('2019-12-31'))
_2020 = (deposit.입금일>=pd.to_datetime('2020-01-01')) & (deposit.입금일<=pd.to_datetime('2020-12-31'))
_2021 = (deposit.입금일>=pd.to_datetime('2021-01-01')) & (deposit.입금일<=pd.to_datetime('2021-12-31'))
_2022 = (deposit.입금일>=pd.to_datetime('2022-01-01')) & (deposit.입금일<=pd.to_datetime('2022-12-31'))
_2023 = (deposit.입금일>=pd.to_datetime('2023-01-01')) & (deposit.입금일<=pd.to_datetime('2023-12-31'))
_2024 = (deposit.입금일>=pd.to_datetime('2024-01-01')) & (deposit.입금일<=pd.to_datetime('2024-12-31'))
_2025 = (deposit.입금일>=pd.to_datetime('2025-01-01')) & (deposit.입금일<=pd.to_datetime('2025-12-31'))

In [33]:
year = 2018
sub_df = deposit[(datetime(year, 1,1) <= deposit.입금일) & (deposit.입금일 <= datetime(year, 12,31))]
sub_df

,주민번호인,계좌키,입금일,입금합계,현재원금
212651,650920-1702917,200923308,2018-12-31,1600000,3849680
212653,630427-1051410,200939783,2018-12-31,204090,3208247
212652,710729-1535419,200934493,2018-12-31,1800000,3937272
212654,600625-1785829,200921145,2018-12-31,300000,22401048
212650,680226-1625812,200921250,2018-12-31,1000000,36822940
...,...,...,...,...,...
237238,740604-1392427,200921331,2018-01-23,300000,49720034
237686,620201-1064111,200921354,2018-01-15,500000,55472550
237935,430320-2930514,200921315,2018-01-10,2000000,45918090
237936,620515-1473643,200921192,2018-01-10,1000000,28515530


In [25]:
# 입금구분 확인건 + 입금자구분
입금구분확인건 = deposit[deposit.입금구분.isin(["부채증명원발급비용","배당금","임시기간회수","기타","법원환급금(수수료)","법착회수","환매대금"])][["채무자키","계좌키","입금키","입금일","입금구분","입금자구분","입금자","입금기간","입금메모","입금비고","입금은행","입금계좌번호","담당자"]]
입금구분확인건.to_excel(join(path_excel, "입금구분.xlsx"), index=False)

In [11]:
종결일 = pd.read_pickle(r"D:\3.자산\전산 dataset\대성\250630\pkl\2.대성_전체_원데이터_250630.pkl")
종결일["종결일"] = pd.to_datetime(종결일.종결일)
종결일 = 종결일[["계좌키","종결일"]].dropna(subset=['종결일'], axis=0)

In [12]:
종결일["계좌키"] = 종결일["계좌키"].astype(str)

In [32]:
deposit.columns

Index(['채무자키', '채권구분', '매각사구분', '입금키', '계좌키', '고객입금일', '입금일', '입금합계', '채무상태',
       '담당자', '입금원금', '입금법비용', '입금미수이자', '입금연체이자', '입금정상이자', '연체일수', '적용이자율',
       '입금기타', '입금자', '입금자구분', '입금구분', '담당입금구분', '입금통계구분', '입금기간', '입금기간일',
       '입금은행', '입금계좌번호', '가상계좌은행', '가상계좌번호', '상대방입금은행', '입금메모', '입금비고', '계좌번호',
       '회수원금', '회수법비용', '회수미수이자', '회수연체이자', '회수기타', '회수합계', '현재원금', '현재법비용',
       '현재미수이자', '현재연체이자', '현재합계', '원가입금', '매출입금', '비용입금', '잡이익', '법조치키',
       '입금고정키', '등록파일', '등록일자', '등록유저', '수정일자', '수정유저', '입금순서', '취소여부', '종결일',
       '매각사', '담보제공처', '담보제공구분', '매입일', '컷오프일', '최초이자기준일', '주민번호인', '지점', '부서',
       '팀', '고객입금액', '채무자명', '타입금키', '타채무자키', '보증인키', '직접입력메모', '담당문자전송',
       '리파이낸싱', '입금수수료율', '매입사'],
      dtype='object')

In [13]:
deposit["입금일"] = pd.to_datetime(deposit["입금일"])

In [14]:
merged = pd.merge(deposit[["채무자키","계좌키","입금키","입금구분","입금일","입금기간","담당자","입금합계"]], 종결일, how='inner')

In [15]:
merged["종결후입금"] = np.where(merged.입금일 <= merged.종결일, "종결전", "종결후")

In [16]:
rst = merged[(merged["종결후입금"] == "종결후") & (merged["입금기간"] != "매각후") & (merged.입금구분!="법원환급금")]

In [ ]:
rst.to_excel(join(path_pkl, "매각후입금.xlsx"), index=False)

In [18]:
deposit[(deposit.입금기간=="원상회복전") & (deposit.입금구분!="신용회복")][["채무자키","계좌키","입금키","입금일","입금자구분","입금구분","입금합계","담당자","채무상태"]].to_excel('원상회복 확인건.xlsx')